In [71]:
import pandas as pd
df = pd.read_csv('anomaly.csv')
print(df.head())

       Date  Power  Detector Quality
0  1-Jan-16     96         8    Good
1  2-Jan-16     96        10    Good
2  3-Jan-16     91         8    Good
3  4-Jan-16     97         9    Good
4  5-Jan-16     91        11    Good


In [72]:
df.drop(['Date'], axis = 1, inplace= True)
df.dropna(inplace=True, axis=1)

In [74]:
print(df.dtypes)

Power        int64
Detector     int64
Quality     object
dtype: object


In [39]:
actual_data = df.copy()
print(actual_data)

      Power  Detector Quality
0        96         8    Good
1        96        10    Good
2        91         8    Good
3        97         9    Good
4        91        11    Good
...     ...       ...     ...
1456     93         8    Good
1457     91        10    Good
1458     95         8    Good
1459     91        10    Good
1460    100        12    Good

[1461 rows x 3 columns]


In [75]:
df.Quality[df.Quality=='Good'] = 1.0
df.Quality[df.Quality=='Bad'] = 2.0
good_mask = df['Quality']== 1.0 #All good to be True for good data points
bad_mask = df['Quality']== 2.0 #All values False for bad data points
df_good = df[good_mask]
df_bad = df[bad_mask]


C:\Users\vijay\AppData\Local\Temp\ipykernel_15908\3576067076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality=='Good'] = 1.0
C:\Users\vijay\AppData\Local\Temp\ipykernel_15908\3576067076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality=='Bad'] = 2.0


In [89]:
df.dtypes
df['Quality'] = df['Quality'].astype('Int64')

In [98]:
random_good = df_good.sample(frac=0.901)  #950
test_good = df_good.drop(random_good.index) #104
random_bad = df_bad.sample(frac=0.123)  #50
test_bad = df_bad.drop(random_bad.index)  #357

df_train = pd.concat([random_good,random_bad],axis=0,ignore_index=True) #1000

#df_train.drop(['Quality'],axis = 1, inplace= True)
df_test = pd.concat([test_good,test_bad],axis=0,ignore_index=True) #461

#df_test.drop(['Quality'],axis = 1, inplace= True)
print(df_train)
print(len(df_test))

     Power  Detector Quality
0       99        11     1.0
1       98        10     1.0
2       98         8     1.0
3       94         8     1.0
4       94         8     1.0
..     ...       ...     ...
995     91         5     2.0
996     75        12     2.0
997     83         9     2.0
998     91         5     2.0
999     85         9     2.0

[1000 rows x 3 columns]
461


In [99]:
df.dtypes

Power       int64
Detector    int64
Quality     Int64
dtype: object

In [100]:
df_train.isnull().sum()

Power       0
Detector    0
Quality     0
dtype: int64

In [101]:
from sklearn import metrics
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [102]:
model = Sequential()
model.add(Dense(10, input_dim=2, activation = 'relu'))
model.add(Dense(3,activation = 'relu'))
model.add(Dense(10,activation = 'relu'))
model.add(Dense(3))
model.compile(loss='mean_squared_error', optimizer = 'adam')
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 10)                30        
                                                                 
 dense_41 (Dense)            (None, 3)                 33        
                                                                 
 dense_42 (Dense)            (None, 10)                40        
                                                                 
 dense_43 (Dense)            (None, 3)                 33        
                                                                 
Total params: 136 (544.00 Byte)
Trainable params: 136 (544.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [103]:
df_train = np.array(df_train)
df_train

array([[99, 11, 1.0],
       [98, 10, 1.0],
       [98, 8, 1.0],
       ...,
       [83, 9, 2.0],
       [91, 5, 2.0],
       [85, 9, 2.0]], dtype=object)

In [104]:
model.fit(df_train,df_train,verbose=1, epochs=100)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
 dense_2 (Dense)             (None, 10)                40        
                                                                 
 dense_3 (Dense)             (None, 2)                 22        
                                                                 
Total params: 125 (500.00 Byte)
Trainable params: 125 (500.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
print(df_train)

     Power  Detector
0       90         9
1       97         8
2       97        11
3       98         8
4       98        11
..     ...       ...
995     99         7
996     93        16
997    112        11
998    113        12
999     94         6

[1000 rows x 2 columns]


In [ ]:
pred_train = model.predict(df_train)
score1 = np.sqrt(metrics.mean_squared_error(pred_train,df_train))

32/32 [==============================] - 0s 1ms/step


In [ ]:
pred_test = model.predict(df_test)
score2 = np.sqrt(metrics.mean_squared_error(pred_test,df_test))

15/15 [==============================] - 0s 2ms/step


In [ ]:

actual_df = pd.concat([df_test,df_train],axis=0,ignore_index=True)
print(df)
print(df_test)
actual_df = actual_df.values

      Power  Detector Quality Detection_1.5 Detection_1.7 Detection_2.0
0        96         8    Good          Good          Good          Good
1        96        10    Good          Good          Good          Good
2        91         8    Good          Good          Good          Good
3        97         9    Good          Good          Good          Good
4        91        11    Good          Good          Good          Good
...     ...       ...     ...           ...           ...           ...
1456     93         8    Good           Bad           Bad          Good
1457     91        10    Good           Bad           Bad           Bad
1458     95         8    Good          Good          Good          Good
1459     91        10    Good          Good          Good          Good
1460    100        12    Good           Bad           Bad           Bad

[1461 rows x 6 columns]
     Power  Detector Detection_1.5 Detection_1.7 Detection_2.0
0     98.0       8.0          Good          Good

In [ ]:
thresholds = [1.5, 1.7, 2.0]
for threshold in thresholds:
    col_name = f'Detection_{threshold}'
    df[col_name] = 'NAN'

for i in range(len(actual_df)):
    input_data = actual_df[i].reshape(1, -1) 
    print(input_data) 
    pred = model.predict(input_data)
    mse = np.sqrt(metrics.mean_squared_error(pred, input_data))
    print(mse)
    

    for threshold in thresholds:
        col_name = f'Detection_{threshold}'
        if mse > threshold:
            df.at[i, col_name] = 'Bad'
        else:
            df.at[i, col_name] = 'Good'

for threshold in thresholds:
    col_name = f'Detection_{threshold}'
    df_test[col_name] = 'NAN'

for i in range(len(df_test)):
    input_data = df_test.values[i].reshape(1, -1) 
    print(input_data) 
    pred = model.predict(input_data)
    mse = np.sqrt(metrics.mean_squared_error(pred, input_data))
    print(mse)
    

    for threshold in thresholds:
        col_name = f'Detection_{threshold}'
        if mse > threshold:
            df_test.at[i, col_name] = 'Bad'
        else:
            df_test.at[i, col_name] = 'Good'
print(df_test)
df.to_csv('test.csv', index=False)

[[98  8]]
1/1 [==============================] - 0s 35ms/step
1.412323343401263
[[96 10]]
1/1 [==============================] - 0s 25ms/step
0.3331042042991043
[[94 12]]
1/1 [==============================] - 0s 24ms/step
0.7852312968357639
[[100  10]]
1/1 [==============================] - 0s 24ms/step
0.5518354899256449
[[98 11]]
1/1 [==============================] - 0s 24ms/step
0.11085200653294777
[[100  11]]
1/1 [==============================] - 0s 26ms/step
0.10093992164567567
[[100   8]]
1/1 [==============================] - 0s 26ms/step
1.526096308747188
[[90  8]]
1/1 [==============================] - 0s 29ms/step
0.9604297436704549
[[98 11]]
1/1 [==============================] - 0s 29ms/step
0.11085200653294777
[[100  10]]
1/1 [==============================] - 0s 25ms/step
0.5518354899256449
[[100  10]]
1/1 [==============================] - 0s 28ms/step
0.5518354899256449
[[98 12]]
1/1 [==============================] - 0s 26ms/step
0.5555812979123342
[[95  8]]
1/1 [==

In [ ]:

predicted_onepointfive = df['Detection_1.5']
predicted_onepointseven = df['Detection_1.7']
predicted_two = df['Detection_2.0']
df.Quality[df.Quality==1] = 'Good'
df.Quality[df.Quality==2] = 'Bad'
print(df)

classification_onepointfive = pd.concat([predicted_onepointfive],axis=1)
classification_onepointseven = pd.concat([predicted_onepointseven],axis=1)
classification_two = pd.concat([predicted_two],axis=1)
print(classification_onepointfive )
print(type(classification_onepointfive ))

      Power  Detector Quality Detection_1.5 Detection_1.7 Detection_2.0
0        96         8    Good          Good          Good          Good
1        96        10    Good          Good          Good          Good
2        91         8    Good          Good          Good          Good
3        97         9    Good          Good          Good          Good
4        91        11    Good          Good          Good          Good
...     ...       ...     ...           ...           ...           ...
1456     93         8    Good           Bad           Bad          Good
1457     91        10    Good           Bad           Bad           Bad
1458     95         8    Good          Good          Good          Good
1459     91        10    Good          Good          Good          Good
1460    100        12    Good           Bad           Bad           Bad

[1461 rows x 6 columns]
     Detection_1.5
0             Good
1             Good
2             Good
3             Good
4             Go

C:\Users\vijay\AppData\Local\Temp\ipykernel_15908\2471883867.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality==1] = 'Good'
C:\Users\vijay\AppData\Local\Temp\ipykernel_15908\2471883867.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality==2] = 'Bad'


In [ ]:
print(df)
print(df_test)



      Power  Detector Quality Detection_1.5 Detection_1.7 Detection_2.0
0        96         8    Good          Good          Good          Good
1        96        10    Good          Good          Good          Good
2        91         8    Good          Good          Good          Good
3        97         9    Good          Good          Good          Good
4        91        11    Good          Good          Good          Good
...     ...       ...     ...           ...           ...           ...
1456     93         8    Good           Bad           Bad          Good
1457     91        10    Good           Bad           Bad           Bad
1458     95         8    Good          Good          Good          Good
1459     91        10    Good          Good          Good          Good
1460    100        12    Good           Bad           Bad           Bad

[1461 rows x 6 columns]
     Power  Detector Detection_1.5 Detection_1.7 Detection_2.0
0     98.0       8.0          Good          Good

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

confusion = confusion_matrix(classification_onepointfive ,df['Quality'])
print(confusion)
sns.heatmap(confusion, 
            annot=True,
            fmt='g', 
            xticklabels=['good', 'bad'],
            yticklabels=['good', 'bad'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix 1.5',fontsize=17)
plt.show()

confusion1 = confusion_matrix(classification_onepointseven ,df['Quality'])
print(confusion)
sns.heatmap(confusion1, 
            annot=True,
            fmt='g', 
            xticklabels=['good', 'bad'],
            yticklabels=['good', 'bad'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix 1.7',fontsize=17)
plt.show()


confusion2 = confusion_matrix(classification_two ,df['Quality'])
print(confusion)
sns.heatmap(confusion1, 
            annot=True,
            fmt='g', 
            xticklabels=['good', 'bad'],
            yticklabels=['good', 'bad'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix 2.0',fontsize=17)
plt.show()